In [2]:
from IPython.display import clear_output
%pip install datasets==2.19.1
%pip install tensorflow==2.15.0
!pip install transformers==4.28.0
%pip install rouge_score
%pip install wandb
%pip install jiwer
%pip install khmer-nltk
clear_output()

In [3]:


print("----- START IMPORTING LIBRARIES -----")

import os
import cv2
import re
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rouge_score
import datasets
import wandb
import string
import random
from khmernltk import word_tokenize

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

from transformers import ViTFeatureExtractor, AutoTokenizer
from transformers import VisionEncoderDecoderModel, XLMRobertaForCausalLM
from transformers import VisionEncoderDecoderConfig
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import default_data_collator
from transformers import EarlyStoppingCallback
from keras.preprocessing.image import ImageDataGenerator

# Log in to Weights & Biases
wandb.login(key="e54bda67bc0e4031b8d6a5b106b28d39585809c4")

# Define model checkpoints
processor_checkpoint = "microsoft/trocr-base-printed"
tokenizer_checkpoint = "channudam/khmer-xlm-roberta-base"

# Load feature extractor and tokenizer
feature_extractor = ViTFeatureExtractor.from_pretrained(processor_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

# Define a punctuation remover
translator = str.maketrans('', '', string.punctuation)

def transform_image(image):
    """Apply a random blur to the input image."""
    rand_int = random.randint(1, 2)
    kernel_size = (rand_int, rand_int)
    image = cv2.blur(image, kernel_size)
    return image
clear_output()
print("----- LIBRARIES SUCCESSFULLY IMPORTED -----")



# print("----- STARTING TO IMPORT LIBRARIES -----")

# import os
# import cv2
# import re
# import torch
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import rouge_score
# import datasets
# import wandb
# import string
# import random
# from khmernltk import word_tokenize

# from sklearn.model_selection import train_test_split
# from torch.utils.data.dataset import Dataset as D

# from transformers import ViTFeatureExtractor, XLMRobertaTokenizer
# from transformers import VisionEncoderDecoderModel, XLMRobertaForCausalLM
# from transformers import VisionEncoderDecoderConfig
# from transformers import DataCollatorForLanguageModeling
# from transformers import Trainer, TrainingArguments
# from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# from transformers import default_data_collator
# from transformers import EarlyStoppingCallback
# from keras.preprocessing.image import ImageDataGenerator

# # Log in to Weights & Biases
# wandb.login(key="e54bda67bc0e4031b8d6a5b106b28d39585809c4")

# # Define model checkpoints
# processor_checkpoint = "microsoft/trocr-base-handwritten"
# tokenizer_checkpoint = "channudam/khmer-xlm-roberta-base"

# # Load feature extractor and tokenizer
# feature_extractor = ViTFeatureExtractor.from_pretrained(processor_checkpoint)
# tokenizer = XLMRobertaTokenizer.from_pretrained(tokenizer_checkpoint)

# # Define a punctuation translator to remove punctuation
# translator = str.maketrans('', '', string.punctuation)

# def transform_image(image):
#     """
#     Apply a random blur to the input image.
#     """
#     rand_int = random.randint(1, 2)
#     kernel_size = (rand_int, rand_int)
#     return cv2.blur(image, kernel_size)

# clear_output()
# print("----- LIBRARIES SUCCESSFULLY IMPORTED -----")

----- LIBRARIES SUCCESSFULLY IMPORTED -----


In [4]:
print("-----START TO CHECK IF GPU(S) ARE AVAILABLE-----")
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

-----START TO CHECK IF GPU(S) ARE AVAILABLE-----
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from khmernltk import word_tokenize

# Read images
print("-----START TO READ IMAGES-----")

image_path = "/kaggle/input/mptc-2k-mptc-data/MPTC-2K-Image"

# Ensure directory exists
if not os.path.exists(image_path):
    print(f"Error: Directory does not exist -> {image_path}")
else:
    # List files and filter PNGs (case-insensitive)
    files = [file for file in os.listdir(image_path) if file.lower().endswith(".png")]
    
    # Read images
    images = np.asarray(
        [plt.imread(os.path.join(image_path, file)) for file in files],
        dtype="object"
    )

    # Extract file names (without extensions) to match with labels
    label_idx = [os.path.splitext(file)[0] for file in files]

    # Read CSV file
    labels_path = "/kaggle/input/mptc-2k-mptc-data/MPTC-2K-Labels.csv"
    if not os.path.exists(labels_path):
        print(f"Error: CSV file not found -> {labels_path}")
    else:
        labels = pd.read_csv(labels_path)

        # Ensure label indices are valid
        if "label" in labels.columns:
            labels_dict = labels.set_index("filename")["label"].to_dict()
            
            # Retrieve labels for available image files
            image_names = [labels_dict.get(file, "") for file in label_idx]

            # Preprocess labels: Remove tabs, punctuation, and extra spaces
            image_names = [re.sub(r"\\t", " ", name) for name in image_names]
            image_names = [name.translate(str.maketrans('', '', string.punctuation)) for name in image_names]
            image_names = [re.sub(r"\s+", " ", name).strip() for name in image_names]
            image_names = [" ".join(word_tokenize(name)) for name in image_names]

            print("-----DONE-----")
            print(f"Total number of images read: {len(images)}")
        else:
            print("Error: CSV file does not contain a 'label' column.")


# # read image
# print("-----START TO READ IMAGES-----")
# image_path = "/kaggle/input/mptc-2k-mptc-data/MPTC-2K-Image"
# files = os.listdir(image_path)
# images = np.asarray(
#     [plt.imread(os.path.join(image_path, file)) for file in files if file.endswith(".jpg")],
#     dtype="object")
# label_idx = [(file.split(".")[0]) for file in files if file.endswith("jpg")]
# labels = pd.read_csv("/kaggle/input/mptc-2k-mptc-data/MPTC-2K-Labels.csv")
# image_names = [re.sub(r"\\t", " ", name) for name in labels.iloc[label_idx]['label'].tolist()]
# image_names = [image_name.translate(translator) for image_name in image_names]
# image_names = [re.sub(r"\s", "", image_name) for image_name in image_names]
# image_names = [" ".join(word_tokenize(image_name)) for image_name in image_names]

# print("-----DONE-----")

-----START TO READ IMAGES-----


| 2025-03-04 04:17:09,728 | INFO | khmer-nltk | Loaded model from /usr/local/lib/python3.10/dist-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav |


-----DONE-----
Total number of images read: 2000


In [6]:
# apply image augmentation
print("-----START TO AUGMENT IMAGES-----")
new_images = []
new_image_names = []
idx = 0
datagen = ImageDataGenerator(
    brightness_range=[0.4,1], 
    preprocessing_function=transform_image,
    fill_mode='nearest')
for image in images:
    new_images.append(image)
    new_image_names.append(image_names[idx])
    raw_img = np.expand_dims(image, axis=0)
    aug_iter = datagen.flow(raw_img, batch_size=1)
    for i in range(1):  #Number of Augment Images
        augmented_images = next(aug_iter)
        augmented_bgr_image = augmented_images[0].astype(np.uint8)
        new_images.append(augmented_bgr_image)
        new_image_names.append(image_names[idx])
    idx = idx+1
new_images = np.asarray(new_images, dtype="object")
new_image_names = np.asarray(new_image_names, dtype="object")
print("-----DONE-----")

-----START TO AUGMENT IMAGES-----
-----DONE-----


In [7]:
print("-----START TO CONVERT DATA TO PANDAS DATAFRAME-----")
df = pd.DataFrame({
    'label': new_image_names,
    'image': [image.flatten() for image in new_images],
    'shape': [image.shape for image in new_images]
})
df.head()
print("-----DONE-----")

-----START TO CONVERT DATA TO PANDAS DATAFRAME-----
-----DONE-----


In [8]:
print("-----START TO SPLIT THE DATASET TO TRAIN TEST-----")
X_train, X_test, y_train, y_test = train_test_split(
df[['image', 'shape']], df['label'], test_size=0.2, random_state=42, stratify=df['label'])
train_df = pd.DataFrame({
    'shape': X_train['shape'],
    'image': X_train['image'],
    'label': y_train,
}).reset_index().drop("index", axis=1)
test_df = pd.DataFrame({
    'shape': X_test['shape'],
    'image': X_test['image'],
    'label': y_test
}).reset_index().drop("index", axis=1)
print(f"train_df shape: {train_df.shape}")
print(f"test_df shape: {test_df.shape}")
print("-----DONE-----")

-----START TO SPLIT THE DATASET TO TRAIN TEST-----
train_df shape: (3200, 3)
test_df shape: (800, 3)
-----DONE-----


In [9]:
print("-----START TO INITIALIZE THE MODEL-----")
# set encoder decoder tying to True
config_encoder = VisionEncoderDecoderConfig.from_pretrained(processor_checkpoint, force_download=True).encoder
config_decoder = XLMRobertaForCausalLM.from_pretrained(tokenizer_checkpoint, is_decoder=True, force_download=True).config
config = VisionEncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder, tie_encoder_decoder=True)
model = VisionEncoderDecoderModel(config)
model.to(device)

-----START TO INITIALIZE THE MODEL-----


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fea

In [10]:
print("-----START TO CONFIG THE MODEL-----")
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.max_length = 128
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4
print("-----DONE-----")


-----START TO CONFIG THE MODEL-----
-----DONE-----


In [11]:
print("-----START TO CONVERT DATAFRAME TO PYTORCH TENSOR-----")

from torch.utils.data import Dataset
import torch
import numpy as np
from PIL import Image

batch_size = 8
max_length = 128

class KhmerTextImageDataset(Dataset):
    def __init__(self, df, tokenizer, feature_extractor, decoder_max_length=max_length):
        self.df = df
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.decoder_max_length = decoder_max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx]['label']  # Ensure correct indexing
        
        # Load the image safely
        image_data = self.df.iloc[idx]['image']
        
        # Ensure image is a NumPy array
        if isinstance(image_data, list):  # Convert nested lists to NumPy arrays
            image_array = np.array(image_data, dtype=np.uint8)
        else:
            image_array = image_data.astype(np.uint8)

        # Convert grayscale images to RGB (H, W, 3)
        if image_array.ndim == 2:  # If 2D (H, W), convert to 3D (H, W, 3)
            image = Image.fromarray(image_array).convert("RGB")  # Convert grayscale to RGB
        else:
            image = Image.fromarray(image_array)  # Already has channels

        # Ensure the image is always in (H, W, 3) format
        image = np.array(image, dtype=np.uint8)

        # Extract pixel values
        pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values

        # Tokenize the label
        labels = self.tokenizer(
            label,
            truncation=True,
            padding="max_length",
            max_length=self.decoder_max_length
        ).input_ids

        # Ensure PAD tokens are ignored in loss computation
        labels = torch.tensor([label if label != self.tokenizer.pad_token_id else -100 for label in labels])

        encoding = {
            "pixel_values": pixel_values.squeeze(0),  # Remove batch dimension
            "labels": labels
        }
        return encoding

# Create train and evaluation datasets
train_dataset = KhmerTextImageDataset(train_df, tokenizer=tokenizer, feature_extractor=feature_extractor)
eval_dataset = KhmerTextImageDataset(test_df, tokenizer=tokenizer, feature_extractor=feature_extractor)

print("-----DONE-----")


# print("-----START TO CONVERT DATAFRAME TO PYTHOCH TENSOR-----")
# from torch.utils.data.dataset import Dataset as D
# batch_size = 8
# max_length = 128
# class KhmerTextImageDataset(D):
#     def __init__(self, df, tokenizer, feature_extractor, decoder_max_length=max_length):
#         self.df = df
#         self.tokenizer = tokenizer
#         self.feature_extractor = feature_extractor
#         self.decoder_max_length = decoder_max_length
#     def __len__(self):
#         return len(self.df)
#     def __getitem__(self, idx):
#         label = self.df['label'][idx]
#         image = self.df['image'][idx].reshape(self.df['shape'][idx])
#         pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values
#         labels = self.tokenizer(label, 
#                                 truncation = True,
#                                 padding="max_length", 
#                                 max_length=self.decoder_max_length).input_ids
#         # important: make sure that PAD tokens are ignored by the loss function
#         labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels]
#         encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
#         return encoding
# train_dataset = KhmerTextImageDataset(train_df,
#                                       tokenizer=tokenizer,
#                                       feature_extractor= feature_extractor)
# eval_dataset = KhmerTextImageDataset(test_df,
#                                      tokenizer=tokenizer,
#                                      feature_extractor= feature_extractor)

# print("-----DONE-----")

-----START TO CONVERT DATAFRAME TO PYTORCH TENSOR-----
-----DONE-----


In [12]:
print("-----START TO INITIALIZE COMPUT METRIC-----")
rouge = datasets.load_metric("rouge", trust_remote_code=True)
cer_metric = datasets.load_metric("cer", trust_remote_code=True)
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    return {
        "cer": cer,
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4)
    }
clear_output()

In [16]:
print("-----START TO CONVERT DATAFRAME TO PYTORCH TENSOR-----")

from torch.utils.data import Dataset
import torch
import numpy as np
from PIL import Image

batch_size = 8
max_length = 128

class KhmerTextImageDataset(Dataset):
    def __init__(self, df, tokenizer, feature_extractor, decoder_max_length=max_length):
        self.df = df
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.decoder_max_length = decoder_max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx]['label']  # Ensure correct indexing
        
        # Load the image safely
        image_data = self.df.iloc[idx]['image']
        
        # Ensure image is a NumPy array
        if isinstance(image_data, list):  # Convert nested lists to NumPy arrays
            image_array = np.array(image_data, dtype=np.uint8)
        else:
            image_array = np.asarray(image_data, dtype=np.uint8)  # Ensure NumPy format

        # ✅ Convert grayscale images to RGB before conversion to NumPy
        image_pil = Image.fromarray(image_array).convert("RGB")  # Convert grayscale (H, W) to RGB (H, W, 3)

        # ✅ Convert PIL image to NumPy array to ensure correct shape (H, W, 3)
        image = np.array(image_pil, dtype=np.uint8)

        # ✅ Ensure image has the correct shape
        if image.ndim == 2:  # If it's still 2D (H, W), add a channel dimension
            image = np.expand_dims(image, axis=-1)  # Convert (H, W) -> (H, W, 1)
            image = np.repeat(image, 3, axis=-1)  # Convert (H, W, 1) -> (H, W, 3)

        # ✅ Pass image inside a list to feature_extractor
        pixel_values = self.feature_extractor([image], return_tensors="pt").pixel_values

        # Tokenize the label
        labels = self.tokenizer(
            label,
            truncation=True,
            padding="max_length",
            max_length=self.decoder_max_length
        ).input_ids

        # Ensure PAD tokens are ignored in loss computation
        labels = torch.tensor([label if label != self.tokenizer.pad_token_id else -100 for label in labels])

        encoding = {
            "pixel_values": pixel_values.squeeze(0),  # Remove batch dimension
            "labels": labels
        }
        return encoding

# Create train and evaluation datasets
train_dataset = KhmerTextImageDataset(train_df, tokenizer=tokenizer, feature_extractor=feature_extractor)
eval_dataset = KhmerTextImageDataset(test_df, tokenizer=tokenizer, feature_extractor=feature_extractor)

print("-----DONE-----")


-----START TO CONVERT DATAFRAME TO PYTORCH TENSOR-----
-----DONE-----


In [ ]:

print("-----START TO SET UP MODEL FINE-TUNING-----")

KhmerOCRModel = "/kaggle/working/KhmerOCRModel-V2"
training_epochs = 20

if not os.path.exists(KhmerOCRModel):
    os.makedirs(KhmerOCRModel)

training_args = Seq2SeqTrainingArguments(
    output_dir=KhmerOCRModel,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    fp16=True,
    evaluation_strategy="epoch",  # FIXED ARGUMENT NAME
    do_train=True,
    do_eval=True,
    logging_steps=1024,  
    save_steps=2048, 
    warmup_steps=1024,  
    num_train_epochs=training_epochs,
    overwrite_output_dir=True,
    save_total_limit=1,
)

trainer = Seq2SeqTrainer(
    tokenizer=feature_extractor,
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

print("-----START TO FINE-TUNING-----")
trainer.train()  # START TRAINING

# print("-----START TO SET UP MODEL FINE-TUNNING-----")

# KhmerOCRModel = "/kaggle/working/KhmerOCRModel"
# training_epochs = 20

# if not os.path.exists(KhmerOCRModel):
#     os.makedirs(KhmerOCRModel)

# training_args = Seq2SeqTrainingArguments(
#     output_dir=KhmerOCRModel,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     predict_with_generate=True,
#     fp16=True,
#     #eval_strategy="epoch",
#     evalution_stragtegy="epoch",
#     do_train=True,
#     do_eval=True,
#     logging_steps=1024,  
#     save_steps=2048, 
#     warmup_steps=1024,  
#     num_train_epochs = training_epochs,
#     overwrite_output_dir=True,
#     save_total_limit=1,
# )

# trainer = Seq2SeqTrainer(
#     tokenizer=feature_extractor,
#     model=model,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     data_collator=default_data_collator,
# )

# print("-----START TO FINE-TUNNING-----")
# # Fine-tune the model, training and evaluating on the train dataset trainer.train()
# trainer.train()

-----START TO SET UP MODEL FINE-TUNING-----
-----START TO FINE-TUNING-----


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Epoch,Training Loss,Validation Loss


In [ ]:
print("-----START TO SAVE THE MODEL-----")
# import zipfile
trainer.save_model(KhmerOCRModel)
feature_extractor.save_pretrained(KhmerOCRModel)
tokenizer.save_pretrained(KhmerOCRModel)
# def zipdir(path, ziph):
#     # ziph is zipfile handle
#     for root, dirs, files in os.walk(path):
#         for file in files:
#             ziph.write(os.path.join(root, file), 
#                        os.path.relpath(os.path.join(root, file), 
#                                        os.path.join(path, '..')))
# with zipfile.ZipFile('KhmerOCRModel.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
#     zipdir('/kaggle/working/KhmerOCRModel', zipf)

In [ ]:
from transformers import TrOCRProcessor
processor = TrOCRProcessor.from_pretrained("/kaggle/working/KhmerOCRModel-V2")
model = VisionEncoderDecoderModel.from_pretrained("/kaggle/working/KhmerOCRModel-V2")
clear_output()

In [ ]:
search_idx = 230
image = test_df['image'][search_idx].reshape(test_df['shape'][search_idx])
print(test_df['label'][search_idx])
plt.imshow(image)
plt.show()

In [ ]:
pixel_values = processor(test_df['image'][search_idx].reshape(test_df['shape'][search_idx]), return_tensors="pt").pixel_values
generated_id = model.generate(pixel_values=pixel_values)
text = processor.batch_decode(generated_id, skip_special_tokens=True)
print(text)